In [87]:
import pickle

In [88]:
with open('../data/assist2009/train_valid_sequences_greed_1.csv_0_1_2_3_4.pkl', 'rb') as f:
    data = pickle.load(f)

In [89]:
data

{'qseqs': tensor([[ 2471,  1402,  5275,  ...,    -1,    -1,    -1],
         [ 4450,  3598, 11922,  ...,  3061,  6790,  7310],
         [ 1599, 11925,  8889,  ..., 16954, 16955,  3300],
         ...,
         [17717, 12868,  5243,  ...,    -1,    -1,    -1],
         [ 8333,  9079, 11361,  ..., 14637, 14128, 14394],
         [14398, 14411, 14420,  ...,    -1,    -1,    -1]], device='cuda:0'),
 'cseqs': tensor([[  5,   5,   2,  ...,  -1,  -1,  -1],
         [  5,   5,   5,  ...,   5,   5,   5],
         [  5,  12,  12,  ...,   5,  90,  12],
         ...,
         [114,   5,   5,  ...,  -1,  -1,  -1],
         [  0,   0,   0,  ...,  71,  71,  71],
         [ 71,  71,  71,  ...,  -1,  -1,  -1]], device='cuda:0'),
 'rseqs': tensor([[ 1.,  1.,  1.,  ..., -1., -1., -1.],
         [ 0.,  0.,  0.,  ...,  0.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  1.,  0.,  ..., -1., -1., -1.],
         [ 1.,  1.,  0.,  ...,  1.,  1.,  0.],
         [ 1.,  1.,  0

In [90]:
import pandas as pd

In [13]:
df = pd.read_csv('../data/assist2009/train_valid_sequences_greed_1.csv', header=None)

In [102]:
df.iloc[31][3]

'0,0,0,0,0,0,0,0,0,1,1,1,1,1,16,16,16,16,16,16,16,16,16,16,17,17,17,16,16,16,16,16,16,16,16,16,16,16,17,17,17,17,3,11,11,11,11,11,11,11,11,11,11,11,11,11,11,58,58,60,53,60,53,60,52,60,52,60,54,60,54,44,45,44,53,44,45,44,45,11,11,11,12,12,12,12,12,12,20,20,20,20,20,16,16,16,17,17,17,17,17,17,17,17,17,17,3,3,48,48,48,48,48,48,48,48,48,48,48,48,48,49,49,49,49,48,48,48,48,48,48,57,57,28,28,28,28,28,20,20,15,20,31,15,15,48,57,57,57,51,57,57,49,71,92,93,94,95,92,93,94,95,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71'

In [103]:
df.iloc[31][3]

'0,0,0,0,0,0,0,0,0,1,1,1,1,1,16,16,16,16,16,16,16,16,16,16,17,17,17,16,16,16,16,16,16,16,16,16,16,16,17,17,17,17,3,11,11,11,11,11,11,11,11,11,11,11,11,11,11,58,58,60,53,60,53,60,52,60,52,60,54,60,54,44,45,44,53,44,45,44,45,11,11,11,12,12,12,12,12,12,20,20,20,20,20,16,16,16,17,17,17,17,17,17,17,17,17,17,3,3,48,48,48,48,48,48,48,48,48,48,48,48,48,49,49,49,49,48,48,48,48,48,48,57,57,28,28,28,28,28,20,20,15,20,31,15,15,48,57,57,57,51,57,57,49,71,92,93,94,95,92,93,94,95,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71'

In [91]:
#!/usr/bin/env python
# coding=utf-8

import os, sys
import pandas as pd
import torch
from torch.utils.data import Dataset
# if torch.cuda.is_available():
#     from torch.cuda import FloatTensor, LongTensor
# else:
# from torch import FloatTensor, LongTensor
import numpy as np


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
else:
    from torch import FloatTensor, LongTensor
# if torch.cuda.is_available():
#     from torch.cuda import FloatTensor, LongTensor
# else:
#     from torch import FloatTensor, LongTensor

class KTDataset(Dataset):
    """Dataset for KT
        can use to init dataset for: (for models except dkt_forget)
            train data, valid data
            common test data(concept level evaluation), real educational scenario test data(question level evaluation).
    Args:
        file_path (str): train_valid/test file path
        input_type (list[str]): the input type of the dataset, values are in ["questions", "concepts"]
        folds (set(int)): the folds used to generate dataset, -1 for test data
        qtest (bool, optional): is question evaluation or not. Defaults to False.
    """
    def __init__(self, file_path, input_type, folds, qtest=False, train_ratio=1.0):
        super(KTDataset, self).__init__()
        self.dori = pd.read_pickle('../data/assist2009/train_valid_sequences_greed_1.csv_0_1_2_3_4.pkl')
        for key in self.dori:
            self.dori[key] = self.dori[key]#[:100]
        print(f"file path: {file_path}, qlen: {len(self.dori['qseqs'])}, clen: {len(self.dori['cseqs'])}, rlen: {len(self.dori['rseqs'])}")

    def __len__(self):
        """return the dataset length
        Returns:
            int: the length of the dataset
        """
        return len(self.dori["rseqs"])

    def __getitem__(self, index):
        """
        Args:
            index (int): the index of the data want to get
        Returns:
            (tuple): tuple containing:
            
            - **q_seqs (torch.tensor)**: question id sequence of the 0~seqlen-2 interactions
            - **c_seqs (torch.tensor)**: knowledge concept id sequence of the 0~seqlen-2 interactions
            - **r_seqs (torch.tensor)**: response id sequence of the 0~seqlen-2 interactions
            - **qshft_seqs (torch.tensor)**: question id sequence of the 1~seqlen-1 interactions
            - **cshft_seqs (torch.tensor)**: knowledge concept id sequence of the 1~seqlen-1 interactions
            - **rshft_seqs (torch.tensor)**: response id sequence of the 1~seqlen-1 interactions
            - **mask_seqs (torch.tensor)**: masked value sequence, shape is seqlen-1
            - **select_masks (torch.tensor)**: is select to calculate the performance or not, 0 is not selected, 1 is selected, only available for 1~seqlen-1, shape is seqlen-1
            - **dcur (dict)**: used only self.qtest is True, for question level evaluation
        """
        dcur = dict()
        mseqs = self.dori["masks"][index]
        for key in self.dori:
            if key in ["masks", "smasks"]:
                continue
            if len(self.dori[key]) == 0:
                dcur[key] = self.dori[key]
                dcur["shft_"+key] = self.dori[key]
                continue
            # print(f"key: {key}, len: {len(self.dori[key])}")
            seqs = self.dori[key][index][:-1] * mseqs
            shft_seqs = self.dori[key][index][1:] * mseqs
            dcur[key] = seqs
            dcur["shft_"+key] = shft_seqs
        dcur["masks"] = mseqs
        dcur["smasks"] = self.dori["smasks"][index]
        # print("tseqs", dcur["tseqs"])
        return dcur

    def __load_data__(self, sequence_path, folds, pad_val=-1,train_ratio=1.0):
        """
        Args:
            sequence_path (str): file path of the sequences
            folds (list[int]): 
            pad_val (int, optional): pad value. Defaults to -1.
        Returns: 
            (tuple): tuple containing
            - **q_seqs (torch.tensor)**: question id sequence of the 0~seqlen-1 interactions
            - **c_seqs (torch.tensor)**: knowledge concept id sequence of the 0~seqlen-1 interactions
            - **r_seqs (torch.tensor)**: response id sequence of the 0~seqlen-1 interactions
            - **mask_seqs (torch.tensor)**: masked value sequence, shape is seqlen-1
            - **select_masks (torch.tensor)**: is select to calculate the performance or not, 0 is not selected, 1 is selected, only available for 1~seqlen-1, shape is seqlen-1
            - **dqtest (dict)**: not null only self.qtest is True, for question level evaluation
        """
        dori = {"qseqs": [], "cseqs": [], "rseqs": [], "tseqs": [], "utseqs": [], "smasks": []}

        # seq_qids, seq_cids, seq_rights, seq_mask = [], [], [], []
        df = pd.read_csv(sequence_path)#[0:1000]
        df = df[df["fold"].isin(folds)]
        interaction_num = 0
        # seq_qidxs, seq_rests = [], []
        for i, row in df.iterrows():
            #use kc_id or question_id as input
            if "concepts" in self.input_type:
                dori["cseqs"].append([int(_) for _ in row["concepts"].split(",")])
            if "questions" in self.input_type:
                dori["qseqs"].append([int(_) for _ in row["questions"].split(",")])
            if "timestamps" in row:
                dori["tseqs"].append([int(_) for _ in row["timestamps"].split(",")])
            if "usetimes" in row:
                dori["utseqs"].append([int(_) for _ in row["usetimes"].split(",")])
                
            dori["rseqs"].append([int(_) for _ in row["responses"].split(",")])
            dori["smasks"].append([int(_) for _ in row["selectmasks"].split(",")])

            interaction_num += dori["smasks"][-1].count(1)

        for key in dori:
            if key not in ["rseqs"]:#in ["smasks", "tseqs"]:
                dori[key] = LongTensor(dori[key])
            else:
                dori[key] = FloatTensor(dori[key])
        # print(f"dori:{dori}")

        mask_seqs = (dori["cseqs"][:,:-1] != pad_val) * (dori["cseqs"][:,1:] != pad_val)
        dori["masks"] = mask_seqs

        dori["smasks"] = (dori["smasks"][:, 1:] != pad_val)
        print(f"interaction_num: {interaction_num}")
        # print("load data tseqs: ", dori["tseqs"])
        return dori
        


In [92]:
curtrain = KTDataset(
            "kdal", "qid", 1 , train_ratio=1)

file path: kdal, qlen: 32, clen: 32, rlen: 32


AttributeError: 'KTDataset' object has no attribute 'data'

In [93]:
from torch.utils.data import DataLoader

In [94]:
train_loader = DataLoader(curtrain, batch_size=8)

In [95]:
device = "cpu" if not torch.cuda.is_available() else "cuda"
for j, data in enumerate(train_loader):
    dcur = data
    q, c, r, t = dcur["qseqs"].to(device), dcur["cseqs"].to(device), dcur["rseqs"].to(device), dcur["tseqs"].to(device)
    qshft, cshft, rshft, tshft = dcur["shft_qseqs"].to(device), dcur["shft_cseqs"].to(device), dcur["shft_rseqs"].to(device), dcur["shft_tseqs"].to(device)
    m, sm = dcur["masks"].to(device), dcur["smasks"].to(device)

    ys, preloss = [], []
    cq = torch.cat((q[:,0:1], qshft), dim=1)
    cc = torch.cat((c[:,0:1], cshft), dim=1)
    cr = torch.cat((r[:,0:1], rshft), dim=1)
    

In [101]:
cc[-2]

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 17, 17, 17, 17,  3, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11, 58, 58, 60, 53, 60, 53, 60, 52, 60, 52, 60, 54, 60, 54, 44,
        45, 44, 53, 44, 45, 44, 45, 11, 11, 11, 12, 12, 12, 12, 12, 12, 20, 20,
        20, 20, 20, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,  3,  3,
        48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 48,
        48, 48, 48, 48, 48, 57, 57, 28, 28, 28, 28, 28, 20, 20, 15, 20, 31, 15,
        15, 48, 57, 57, 57, 51, 57, 57, 49, 71, 92, 93, 94, 95, 92, 93, 94, 95,
        71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,
        71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71,
        71, 71], device='cuda:0')

tensor([48, 48, 50, 58, 58, 59, 61, 61, 61, 61, 61, 61, 61, 61, 61, 15, 61, 15,
        61, 15, 61, 15, 61, 15, 61, 15, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 64, 62, 62, 62, 62, 62, 62, 62, 63, 63,
        63, 63, 63, 63, 63, 63, 63, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 65, 66, 66, 62, 67, 68, 67, 68, 67, 68, 67, 68, 67, 68,
        67, 68, 67, 68, 67, 68, 67, 68, 67, 68, 56, 67, 68, 55, 56, 55, 56, 55,
        56, 55, 56, 55, 56, 55, 56, 56, 69, 70, 70, 70, 70, 55, 56, 55, 56, 55,
        56, 55, 56, 55, 56, 55, 56, 62, 62, 62, 62, 71, 71, 71, 55, 56, 55, 56,
        55, 56, 48, 48, 48, 48, 48, 49, 48, 49, 48, 49, 48, 49, 48, 49, 48, 49,
        48, 69, 69, 69, 69, 69, 69, 72, 73, 73, 73, 73, 73, 48, 48, 48, 48, 48,
        48, 49, 48, 49, 48, 49, 48, 49, 48, 49, 48, 57, 57, 57, 57, 57, 57, 51,
        57, 51], device='cuda:0')